### Магический метод new

Магический метод new вызывается перед созданием экземпляра класса, а метод init - после создания объекта класса.

В методе new обязательным параметром является cls, а в методе init - self.<br> cls является ссылкой на класс, а self - ссылкой на экземпляр класса.

In [1]:
class Point:
    def __new__(cls, *args, **kwargs):
        print(f'Вызов метода new для {str(cls)}')
    
    def __init__(self, x=0, y=0):
        print(f'Вызов метода init для {str(self)}')
        self.x = x
        self.y = y

Если попробовать создать объект, будет вызван только метод new:

In [2]:
pt = Point(1, 2)

Вызов метода new для <class '__main__.Point'>


Произошло это потому, что в данный момент экземпляр класса не был создан. В этом можно убедится если посмотреть, на что ссылается ссылка pt:

In [3]:
print(pt)

None


None говорит о том, что экземпляр не был создан. Дело в том, что магический метод new должен возвращать адрес нового созданного объекта, а в нашем случае он ничего не возвращает.  

Для того, чтобы это произошло, нужно вызвать аналогичный метод new из базового класса:

In [4]:
class Point:
    def __new__(cls, *args, **kwargs):
        print(f'Вызов метода new для {str(cls)}')
        return super().__new__(cls)
    
    def __init__(self, x=0, y=0):
        print(f'Вызов метода init для {str(self)}')
        self.x = x
        self.y = y

super() - ссылка на базовый класс, из которого был вызван метод new, которому была передана ссылка на текущий класс Point. Начиная с версии Python 3 все классы автоматически и неявно наследуются от базового класса object. И именно из этого базового класса мы вызываем метод new. То есть когда мы вызываем функцию super в классе Point, мы получаем ссылку на базовый класс, и в этом базовом классе вызываем машический метод new, который и запускает процесс создания экземпляра класса и возвращает адрес нового созданного объекта.

Теперь при создании экземпляра класса сначала вызывается метод new, а затем - метод init. А сама переменная pt ссылается на экземпляр класса Point:

In [5]:
pt = Point(1, 2)
print(pt)

Вызов метода new для <class '__main__.Point'>
Вызов метода init для <__main__.Point object at 0x0000023A183C2F40>


args и kwargs нужны для того, чтобы принимать аргументы, которые передаются при создании экземпляра класса. Перед init они сначала попадают в new, и если убрать args и kwargs - то будет ошибка. Более того, эти аргументы можно использовать в самом методе new, чтобы реализовать какую либо логику.

### Паттерн Singleton

Предположим, что существует класс DataBase, и в программе мы подразумеваем, что может существовать только один экземпляр данного класса. Т.е. два экземпляра класса в один момент существовать не может.

Для реализации этого паттерна можно поступить следующим образом:

Пропишем в классе DataBase один атрибут класса - instance. Он будет принимать значение None. Instance будет ссылкой на экземпляр класса. Если его нет - этот атрибут будет принимать значение None, а если он будет - то будет содержать ссылку на экземпляр класса. Таким образом можно будет контролировать, существует ли объект у этого класса или нет. А для управления созданием новых объектов нужно переопределить метод new: в случае, если instance принимает значение None, то атрибуту instance будет присвоен адрес нового созданного объекта, после чего метод new вернет адрес на этот объект. А в случае, если такой объект существует (instance не принимает значение None) - то будет просто возвращена ссылка на уже существующий объект. Таким образом будет гарантированно, что в программе существует ровно один объект класса DataBase. В финализаторе del, который запускается перед удалением объекта, необходимо указать, чтобы параметр instance принимал значение None в случае, если единственный объект будет удален. Таким образом можно будет повторно создать объект после его удаления:

In [6]:
class DataBase:
    __instance = None
    
    def __new__(cls, *args, **kwargs):
        print('Вызов new')
        if cls.__instance is None:
            cls.__instance = super().__new__(cls)
            
        return cls.__instance
    
    def __del__(self):
        print('Вызов del')
        DataBase.__instance = None
        
    def __init__(self, user, psw, port):
        print('Вызов init')
        self.user = user
        self.psw = psw
        self.port = port
    
    def connect(self):
        print(f'Соединение с БД: {self.user}, {self.psw}, {self.port}')
        
    def close(self):
        print('Закрытие соединения с базой данных')
    
    def read(self):
        return 'Данные из базы данных'
    
    def write(self, data):
        print(f'Запись информации {data} в базу данных')

Пропробуем создать два экземпляра класса:

In [7]:
db1 = DataBase('default_user', 'default_psw', 4040)

Вызов new
Вызов init


In [8]:
db2 = DataBase('default_user', 'default_psw', 4044)

Вызов new
Вызов init


А потом выведем их id:

In [9]:
id(db1)

2448537663856

In [10]:
id(db2)

2448537663856

id совпадают - это означает, что переменные db1 и db2 ссылаются на один объект:

In [11]:
id(db1) == id(db2)

True

Это означает, что при попытке создания второго объекта он на самом деле не был создан, и переменная db2 ссылается на тот же объект, что был создан ранее - db1.

Но если вызвать метод connect у этих объектов, будет видно, что оба содержат параметры, которые были переданы при попытке создания второго экземпляра:

In [12]:
db1.connect()

Соединение с БД: default_user, default_psw, 4044


In [13]:
db2.connect()

Соединение с БД: default_user, default_psw, 4044


По хорошему, если ранее был создан объект, то должны оставаться те параметры, с которыми он был создан (если они явно не были изменены). Это происходит потому, что метод init во второй раз вызывается и меняет параметры в соответствии с новыми значениями. Это можно исправить, если дополнительно переопределить метод call, чтобы init при попытке создания второго объекта не вызывался.